In [53]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("kaggle_eegs").getOrCreate()


In [54]:
file_path = '/home/harraz/my_tensorflow/venv/eegs/training_data/train.csv'

# Read labels CSV file into a DataFrame
train_y = spark.read.csv(file_path, header=True, inferSchema=True)

# Show the DataFrame
train_y.show(10)


+----------+----------+------------------------+--------------+------------------+--------------------------------+----------+----------+----------------+------------+--------+--------+---------+---------+----------+
|    eeg_id|eeg_sub_id|eeg_label_offset_seconds|spectrogram_id|spectrogram_sub_id|spectrogram_label_offset_seconds|  label_id|patient_id|expert_consensus|seizure_vote|lpd_vote|gpd_vote|lrda_vote|grda_vote|other_vote|
+----------+----------+------------------------+--------------+------------------+--------------------------------+----------+----------+----------------+------------+--------+--------+---------+---------+----------+
|1628180742|         0|                     0.0|        353733|                 0|                             0.0| 127492639|     42516|         Seizure|           3|       0|       0|        0|        0|         0|
|1628180742|         1|                     6.0|        353733|                 1|                             6.0|3887563113|     4

In [74]:
from pyspark.sql import DataFrame

file_path = '/home/harraz/my_tensorflow/venv/eegs/training_data/*.parquet'

# Read parquet file into a DataFrame
train_X = spark.read.parquet(file_path, inferSchema=False)


In [56]:
train_X.show(5)

+------+------+------+-----+------+------+------+------+-----+-----+-----+------+------+------+-----+------+------+-----+-----+-----+
|   Fp1|    F3|    C3|   P3|    F7|    T3|    T5|    O1|   Fz|   Cz|   Pz|   Fp2|    F4|    C4|   P4|    F8|    T4|   T6|   O2|  EKG|
+------+------+------+-----+------+------+------+------+-----+-----+-----+------+------+------+-----+------+------+-----+-----+-----+
|-42.11|-29.85|-11.75|  0.9|-46.79|-58.51|-46.76| -3.15| 1.94|44.35|24.83|-33.15|-61.54|-16.39|16.18|-56.23|-37.86| -6.5|-6.31|-5.63|
|-37.88|-26.68|-12.61|-1.25|-43.48|-58.28|-49.03| -5.92| 3.91|44.44|21.42|-28.72|-55.35|-15.21|14.24| -49.8| -39.5| -7.0|-6.44|-1.36|
|-44.89|-28.87| -13.7|-4.53|-44.32|-60.15|-51.88|-10.11|  1.0|45.67|18.26|-33.62|-51.27|-15.53|12.69|-46.76|-33.41|-7.38|-7.63| 1.18|
|-30.95|-21.98|-10.18|-4.15| -38.4|-57.27|-50.53|-11.32| 7.41|48.78|18.54|-19.05|-40.24|-12.39|13.28|-37.09|-27.82|-8.45| -8.8|-4.78|
|-16.65|-11.68| -4.09|-0.36|-30.12|-50.98| -46.5| -7.77|15.96|

In [58]:
from pyspark.sql.functions import when

train_Labels = train_y.withColumn('expert_consensus', when(train_y['expert_consensus'] == 'Seizure', 1).otherwise(0))
train_Labels.select(train_Labels.expert_consensus).show(10)


+----------------+
|expert_consensus|
+----------------+
|               1|
|               1|
|               1|
|               1|
|               1|
|               1|
|               1|
|               1|
|               1|
|               0|
+----------------+
only showing top 10 rows



In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler


# Extract feature columns (excluding 'id')
feature_columns = [col_name for col_name in train_X.columns if col_name != 'id'] 

from pyspark.sql.types import StructType, StructField, FloatType

tmp=train_X

# Assemble features into a single vector column
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features_assembled")
train_X = assembler.transform(train_X)

# Create a StandardScaler instance
scaler = StandardScaler(inputCol="features_assembled", outputCol="normalized_features", withStd=True, withMean=True)

# # Fit and transform the DataFrame
# scaler_model = scaler.fit(train_X)
# normalized_X = scaler_model.transform(train_X)

# # Show the results
# normalized_X.select("id", "features", "normalized_features").show(truncate=False)


In [77]:
scaler_model = scaler.fit(train_X)


24/01/31 12:55:57 ERROR Executor: Exception in task 0.0 in stage 62.0 (TID 62073)
org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`VectorAssembler$$Lambda$3713/0x00007fe3e90ab298`: (struct<Fp1_double_VectorAssembler_51fde68db329:double,F3_double_VectorAssembler_51fde68db329:double,C3_double_VectorAssembler_51fde68db329:double,P3_double_VectorAssembler_51fde68db329:double,F7_double_VectorAssembler_51fde68db329:double,T3_double_VectorAssembler_51fde68db329:double,T5_double_VectorAssembler_51fde68db329:double,O1_double_VectorAssembler_51fde68db329:double,Fz_double_VectorAssembler_51fde68db329:double,Cz_double_VectorAssembler_51fde68db329:double,Pz_double_VectorAssembler_51fde68db329:double,Fp2_double_VectorAssembler_51fde68db329:double,F4_double_VectorAssembler_51fde68db329:double,C4_double_VectorAssembler_51fde68db329:double,P4_double_VectorAssembler_51fde68db329:double,F8_double_VectorAssembler_51fde68db329:double,T4_double_VectorAssembler_

Py4JJavaError: An error occurred while calling o991.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 62.0 failed 1 times, most recent failure: Lost task 0.0 in stage 62.0 (TID 62073) (192.168.1.155 executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`VectorAssembler$$Lambda$3713/0x00007fe3e90ab298`: (struct<Fp1_double_VectorAssembler_51fde68db329:double,F3_double_VectorAssembler_51fde68db329:double,C3_double_VectorAssembler_51fde68db329:double,P3_double_VectorAssembler_51fde68db329:double,F7_double_VectorAssembler_51fde68db329:double,T3_double_VectorAssembler_51fde68db329:double,T5_double_VectorAssembler_51fde68db329:double,O1_double_VectorAssembler_51fde68db329:double,Fz_double_VectorAssembler_51fde68db329:double,Cz_double_VectorAssembler_51fde68db329:double,Pz_double_VectorAssembler_51fde68db329:double,Fp2_double_VectorAssembler_51fde68db329:double,F4_double_VectorAssembler_51fde68db

In [60]:
# from pyspark.sql.types import StructType, StructField, DoubleType

# # Define the schema struct with DoubleType
# schema_struct_double = StructType([
#     StructField("Fp1", DoubleType(), True),
#     StructField("F3", DoubleType(), True),
#     StructField("C3", DoubleType(), True),
#     StructField("P3", DoubleType(), True),
#     StructField("F7", DoubleType(), True),
#     StructField("T3", DoubleType(), True),
#     StructField("T5", DoubleType(), True),
#     StructField("O1", DoubleType(), True),
#     StructField("Fz", DoubleType(), True),
#     StructField("Cz", DoubleType(), True),
#     StructField("Pz", DoubleType(), True),
#     StructField("Fp2", DoubleType(), True),
#     StructField("F4", DoubleType(), True),
#     StructField("C4", DoubleType(), True),
#     StructField("P4", DoubleType(), True),
#     StructField("F8", DoubleType(), True),
#     StructField("T4", DoubleType(), True),
#     StructField("T6", DoubleType(), True),
#     StructField("O2", DoubleType(), True),
#     StructField("EKG", DoubleType(), True)
# ])
# # Use the VectorAssembler with explicit input and output columns
# assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
# train_X = assembler.transform(train_X)


In [61]:
train_X.show(truncate=False)

+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-----+
|Fp1   |F3    |C3    |P3    |F7    |T3    |T5    |O1    |Fz    |Cz    |Pz    |Fp2   |F4    |C4    |P4    |F8    |T4    |T6    |O2    |EKG  |
+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-----+
|-42.11|-29.85|-11.75|0.9   |-46.79|-58.51|-46.76|-3.15 |1.94  |44.35 |24.83 |-33.15|-61.54|-16.39|16.18 |-56.23|-37.86|-6.5  |-6.31 |-5.63|
|-37.88|-26.68|-12.61|-1.25 |-43.48|-58.28|-49.03|-5.92 |3.91  |44.44 |21.42 |-28.72|-55.35|-15.21|14.24 |-49.8 |-39.5 |-7.0  |-6.44 |-1.36|
|-44.89|-28.87|-13.7 |-4.53 |-44.32|-60.15|-51.88|-10.11|1.0   |45.67 |18.26 |-33.62|-51.27|-15.53|12.69 |-46.76|-33.41|-7.38 |-7.63 |1.18 |
|-30.95|-21.98|-10.18|-4.15 |-38.4 |-57.27|-50.53|-11.32|7.41  |48.78 |18.54 |-19.05|-40.24|-12.39|13.28 |-37.09|-27.82|-8.45 |-8.8  |-4.78|
|-16.65|-11.6

In [62]:
from pyspark.sql.types import ArrayType, DoubleType
from pyspark.sql.functions import udf, col
import numpy as np

# Define a UDF for normalization using numpy
@udf(ArrayType(DoubleType()))
def normalize_features(*args):
    numeric_values = [x if isinstance(x, (int, float)) and not np.isnan(x) else 0 for x in args]
    
    if len(numeric_values) > 0:
        mean_value = np.mean(numeric_values)
        normalized_features = [(x - mean_value) / np.std(numeric_values) if isinstance(x, (int, float)) else x for x in args]
        return normalized_features
    else:
        return args

# Extract column names from the DataFrame
feature_columns = train_X.columns

# Apply the UDF to normalize the features
# df = train_X.withColumn("normalized_features", normalize_features(*[col(col_name) for col_name in feature_columns]))

normalized_df = train_X

# Apply the UDF to each column separately
for column_name in feature_columns:
    normalized_df = normalized_df.withColumn(column_name, normalize_features(col(column_name)))


# Show the DataFrame with the normalized features
normalized_df.show(truncate=False)



/tmp/ipykernel_13024/1614526593.py:12: RuntimeWarning: invalid value encountered in scalar divide
/tmp/ipykernel_13024/1614526593.py:12: RuntimeWarning: invalid value encountered in scalar divide
/tmp/ipykernel_13024/1614526593.py:12: RuntimeWarning: invalid value encountered in scalar divide
/tmp/ipykernel_13024/1614526593.py:12: RuntimeWarning: invalid value encountered in scalar divide
/tmp/ipykernel_13024/1614526593.py:12: RuntimeWarning: invalid value encountered in scalar divide
/tmp/ipykernel_13024/1614526593.py:12: RuntimeWarning: invalid value encountered in scalar divide
/tmp/ipykernel_13024/1614526593.py:12: RuntimeWarning: invalid value encountered in scalar divide
/tmp/ipykernel_13024/1614526593.py:12: RuntimeWarning: invalid value encountered in scalar divide
/tmp/ipykernel_13024/1614526593.py:12: RuntimeWarning: invalid value encountered in scalar divide
/tmp/ipykernel_13024/1614526593.py:12: RuntimeWarning: invalid value encountered in scalar divide
/tmp/ipykernel_13024

Py4JJavaError: An error occurred while calling o882.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 54.0 failed 1 times, most recent failure: Lost task 0.0 in stage 54.0 (TID 52064) (192.168.1.155 executor driver): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for numpy.dtype). This happens when an unsupported/unregistered class is being unpickled that requires construction arguments. Fix it by registering a custom IObjectConstructor for this class.
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:759)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:199)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:109)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:122)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.$anonfun$evaluate$2(BatchEvalPythonExec.sc

In [ ]:
from pyspark.sql.functions import count, when, isnan

# Identify non-numeric values in each column
non_numeric_counts = train_X.select([count(when(~col(c).cast("double").isNotNull(), c)).alias(c) for c in train_X.columns])

# Show the counts of non-numeric values for each column
non_numeric_counts.show(truncate=False)


+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+
|Fp1   |F3    |C3    |P3    |F7    |T3    |T5    |O1    |Fz    |Cz    |Pz    |Fp2   |F4    |C4    |P4    |F8    |T4    |T6    |O2    |EKG   |
+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+
|345211|345211|345211|345211|345211|345211|345211|345211|345211|345211|345211|345211|345211|345211|345211|345211|345211|345211|345211|345211|
+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+



In [ ]:
non_numeric = train_X.select([when(~col(c).cast("double").isNotNull(), c).alias(c) for c in train_X.columns])
non_numeric.show()

+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
| Fp1|  F3|  C3|  P3|  F7|  T3|  T5|  O1|  Fz|  Cz|  Pz| Fp2|  F4|  C4|  P4|  F8|  T4|  T6|  O2| EKG|
+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|
|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|
|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|
|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|
|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|
|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|
|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|N